In [1]:
import ROOT, rat
import os, sys, pickle

import matplotlib.pyplot as plt
import numpy as np
import collections
%matplotlib inline 

import jp_mpl as jplot
import geo_studies, rat_misc

from scipy import optimize
reload(rat_misc)
from matplotlib.colors import LogNorm
import pickle

# My tools
import jp_analysis as analysis


TClass::TClass:0: RuntimeWarning: no dictionary for class omtext is available


In [2]:
# Airplane mode
db = rat.RAT.DB.Get()
db.SetAirplaneModeStatus(True)
db.SetDefaultPlaneLockStatus(False)
print 'This is AIRPLANE MODE - be careful!'

This is AIRPLANE MODE - be careful!


TClass::TClass:0: RuntimeWarning: no dictionary for class RAT::DBTableKey is available


In [3]:
socdir = '/sb/project/qbs-015-ac/jpyanez/data/SOC_files'
peakdir = '/sb/project/qbs-015-ac/jpyanez/data/SOC_peak_files_gauss'

In [4]:
npmts = 9728

In [ ]:
# Function to find the peak

In [5]:
def getTimeDiff(time_array,
                bin_width=0.5, # in ns
                peak_width=4., # in ns 
                peak_width2 = 6.,
                
                expected_tdelay = 76.,
                plot = False,
                debug = False):
    
    time_array = time_array[time_array>0]
    
    #### Gaussian based algorithm ####
    
    # Bin the ToA
    ybins  = np.linspace(time_array.mean()-50., 
                         time_array.mean()+150, 
                         (200/bin_width)+1)
    
    # ycenters
    ycenters = (ybins[1:] + ybins[:-1])/2.
    
    n, x = np.histogram(time_array, ybins)
    
    
    
    # Find the highest peak
    prompt_peak_index = n.argmax()
    
    # First peak range (in bins)
    fit_s = prompt_peak_index - int(peak_width/bin_width)
    fit_e = prompt_peak_index + int(peak_width/bin_width)
    
    
    
    # Fit a gaussian +/- 4. ns around the peak
    popt =  [n[prompt_peak_index]/2., ycenters[prompt_peak_index],3.]
    try:
        popt, pcov = optimize.curve_fit(analysis.gaus,
                                        ycenters[fit_s:fit_e+1],
                                        n[fit_s:fit_e+1],
                                        popt,
                                       )
        perr = np.sqrt(np.diag(pcov))
        terr1 = perr[1]
        t1 = popt[1]
    except:
        terr1 = -1
        t1 = -1
    
    # Fit a gaussian around the second peak
    refl_peak_index = prompt_peak_index + int(expected_tdelay/bin_width)
    fit_s2 = refl_peak_index - int(peak_width2/bin_width)
    fit_e2 = refl_peak_index + int((peak_width2*2)/bin_width)
    fit_e2 = np.min([fit_e2, ybins.size-1])
    
    if refl_peak_index > (ybins.size-1):
        terr2 = -1
        t2 = -1
    else:   
        # Fit a gaussian +/- 4. ns around the peak
        try:
            popt2 = [n[refl_peak_index]/2.,ycenters[refl_peak_index],3.]
            popt2, pcov2 = optimize.curve_fit(analysis.gaus,
                                              ycenters[fit_s2:fit_e2+1],
                                              n[fit_s2:fit_e2+1],
                                              popt2)   
            perr2 = np.sqrt(np.diag(pcov2))
            terr2 = perr2[1]
            t2 = popt2[1]
        except RuntimeError: 
            terr2 = -1
            t2 = -1
    

    if plot:
        fig = plt.figure(figsize=(7,4))
        ax1 = fig.add_subplot(111)
        jplot.unfilledBar(x, n, color='0.7')

        newx = np.linspace(ycenters[fit_s]-3., ycenters[fit_e]+3., 101)
        plt.plot(newx, analysis.gaus(newx, *popt), 'r')
        
        try:
            newx = np.linspace(ycenters[fit_s2]-3., ycenters[fit_e2]+3., 101)
            plt.plot(newx, analysis.gaus(newx, *popt2), 'r')    
            plt.xlim([ycenters[prompt_peak_index]-20., ycenters[refl_peak_index] + 20])
        except:
            print 'Could not plot second gaus (out of bounds)'
        
        plt.yscale('log')
        plt.ylabel('N hits')
        plt.ylim([1., n[prompt_peak_index]*1.3])

        return fig
    
        #raw_input()
    return t1, terr1, t2, terr2

In [6]:
infile_list = os.listdir(socdir)
#for i, x in enumerate(infile_list): print i, x 

## Testing a handful of PMT for all files - prerequisite!

 ## Verify the output before moving forward!

In [7]:
# Sadly, the verification is done by hand
usable_run_list =  [17375,
100556,
100558,
101427,
101428,
101432,
101433,
102518,
102529,
102552,
102554,
102570,
102572,
102574]

In [8]:
# Remove useless runs from file list
original_infile_list = os.listdir(socdir)
infile_list = []
for one_file in original_infile_list:
    for usable_run in usable_run_list:
        if "%i" % usable_run in one_file:
            infile_list.append(one_file)
            break
            
print 'List of good run files'
infile_list

List of good run files


['SOC_0000017375_reprocessed.root',
 'SOC_0000100556.root',
 'SOC_0000100558.root',
 'SOC_0000101427_reprocessed_newPCA.root',
 'SOC_0000101428_reprocessed_newPCA.root',
 'SOC_0000101432_reprocessed_newPCA.root',
 'SOC_0000101433_reprocessed_newPCA.root',
 'SOC_0000102518.root',
 'SOC_0000102529.root',
 'SOC_0000102552.root',
 'SOC_0000102554.root',
 'SOC_0000102570.root',
 'SOC_0000102572.root',
 'SOC_0000102574_000.root',
 'SOC_0000102574_001.root',
 'SOC_0000102574_002.root',
 'SOC_0000102574_003.root']

## Running over all the files

In [9]:
bin_width = 0.5

In [ ]:
while len(infile_list) > 0:
    one_file = infile_list[0]
    data = None
    # Need to establish if the file has subfiles
    run_number = one_file[:14]
    new_list   = []
    for one_name in infile_list:
        if run_number in one_name:
            new_list.append(one_name)
    for one_name in new_list:
        infile_list.remove(one_name)
        
    
    #print run_number
    #print 'New list', new_list
    #print infile_list
    #continue
            
    reader_list = []
    soc_list = []
    zero_counter = 0
    skip_counter = 0
    soc_pmts = np.array([0])
    already_processed = False
    for i, this_name in enumerate(new_list):
        infile_name = os.path.join(socdir, this_name)
        
        if i == 0:
            print '\nRun', this_name, ' - Subruns', len(new_list)
            outfile_name = os.path.join(peakdir, run_number + '.pckl')
            if os.path.isfile(outfile_name):
                print 'Run already processed. Skipping it ...'
                already_processed = True
                break
            else:
                print outfile_name
                
        
        # Need to open all the readers
        reader_list.append(rat.socreader(infile_name))
        this_soc, this_run = reader_list[-1].next()
        soc_list.append(this_soc)
        soc_pmts = np.concatenate((soc_pmts, np.array(this_soc.GetSOCPMTIDs())))

        if i == 0:
            # Get the basic info from the first file
            manip_pos = np.array(this_soc.calib.GetPos())
            fit_pos  = np.array(this_soc.GetFitResult(this_soc.GetFitNames()[0]).GetVertex(0).GetPosition())

            data = {'manip_pos':manip_pos,
                    'fit_pos':fit_pos,
                    'soc_pmts':soc_pmts}
    
            delta_ts = np.zeros([npmts, 4])
            poccupancy = np.zeros(npmts)
            print 'Going over the ', len(soc_pmts), ' PMTs'
                              
    if already_processed:
        continue     

    soc_pmts = np.unique(soc_pmts)
        
    for ipmt, one_pmt in enumerate(soc_pmts):
        time_array = np.array([0])
        for isoc, soc in enumerate(soc_list):
            try:
                poccupancy[one_pmt] += soc.GetSOCPMT(one_pmt).GetPromptOccupancy()

                time_array = np.concatenate((time_array,
                                             np.array(soc.GetSOCPMT(one_pmt).GetTimes())))

                time_array[time_array < 0] = 0.
                if time_array.mean() == 0:
                    skip_counter += 1
                    continue
            except:
                print 'PMT problematic', one_pmt, new_list[isoc]
                continue
                
            if ipmt % 500 == 0:
                nonzeros = (delta_ts[:,2] > 0)*(delta_ts[:,0] > 0)
                print ipmt, (delta_ts[nonzeros,2]-delta_ts[nonzeros,0]).mean()
                
                
        if time_array.mean() == 0 or np.sum(time_array>0) < 1000:
            continue

            
        # Avoid placing this command in a "try" 
        delta_ts[one_pmt, :] = getTimeDiff(time_array,
                                                bin_width = bin_width)
        if np.sum(delta_ts[one_pmt,:]) == 0:
            zero_counter += 1

    data['poccupancy'] = poccupancy
    data['delta_ts'] = delta_ts
    print 'Zero counter', zero_counter
    print 'Skip counter', skip_counter
    
    pickle.dump(data, open(outfile_name,'w'))
    
    for reader in reader_list:
        reader.close()


Run SOC_0000017375_reprocessed.root  - Subruns 1
/sb/project/qbs-015-ac/jpyanez/data/SOC_peak_files_gauss/SOC_0000017375.pckl
Going over the  9256  PMTs
PMT problematic 0 SOC_0000017375_reprocessed.root
1000 73.4191631449
1500 73.1683719265
2000 73.0932011045
2500 72.9336680894
3000 72.8875597865
3500 72.8976077157
4000 72.9232614014
4500 72.9889911895
5000 73.1535145638
5500 73.3061501683
6000 73.4616073061
6500 73.5973559291
7000 73.7434089719
7500 73.8724015213
8000 73.9310284155
8500 73.9983962712
9000 74.0398390352
Zero counter 0
Skip counter 546

Run SOC_0000100556.root  - Subruns 1
/sb/project/qbs-015-ac/jpyanez/data/SOC_peak_files_gauss/SOC_0000100556.pckl
Going over the  9196  PMTs
1000 74.1509000703
2000 73.988526449
2500 73.8202646814
3000 73.7695843452
3500 73.7894245402
4000 73.7993630521
4500 73.8638724894
5000 74.0011109808
5500 74.1383926629
6000 74.2759788776
6500 74.3955304161
7000 74.5281643198
7500 74.6445251481
8000 74.7101604306
8500 74.7692756047
9000 73.6084521

/software/CentOS-6/eb/software/Toolchain/iomkl/2015b/Python/2.7.10/lib/python2.7/site-packages/scipy/optimize/minpack.py:604: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/gs/software/CentOS-6/eb/software/Toolchain/iomkl/2015b/IPython/3.2.1-Python-2.7.10/lib/python2.7/site-packages/IPython/kernel/__main__.py:65: RuntimeWarning: invalid value encountered in sqrt


 110.044282066
1500 111.324189
3000 120.012148723
3500 121.05504031
4000 123.255742458
4500 121.790449174
5000 116.639112834
5500 117.332497857
6000 121.08300278
6500 119.72211778
7000 118.889821286
7500 118.100381849
8000 119.924440537
8500 118.051077512
9000 117.429962114
Zero counter 0
Skip counter 571

Run SOC_0000102572.root  - Subruns 1
/sb/project/qbs-015-ac/jpyanez/data/SOC_peak_files_gauss/SOC_0000102572.pckl
Going over the  9309  PMTs
PMT problematic 0 SOC_0000102572.root
500 114.463989733
1000 115.287667474
1500 124.542890002
2500 119.332095168
3000 118.336789992
3500 119.223174204
4000 119.040802009
4500 114.803778867
5000 116.70301264
5500 119.50703872
6000 118.527983806
6500 121.82864608
7000 121.664707528
7500 118.631042665
8000 119.03325823
8500 120.352467967
9000 118.972537118
Zero counter 0
Skip counter 561

Run SOC_0000102574_000.root  - Subruns 4
/sb/project/qbs-015-ac/jpyanez/data/SOC_peak_files_gauss/SOC_0000102574.pckl
Going over the  9303  PMTs
PMT problematic 1

In [ ]:
print 'Hey'